In [6]:
import pandas as pd
import geopandas as gpd
import os
import overpass

Download the [OS Open Names dataset](https://osdatahub.os.uk/data/downloads/open/OpenNames) and extract the zip into this directory, it should produce a directory called `opname_csv_gb`.

In [74]:
with open("opname_csv_gb/Doc/OS_Open_Names_Header.csv") as f:
    header = f.readline().split(',')

results = pd.DataFrame()
with os.scandir("opname_csv_gb/Data") as it:
    for entry in it:
        if entry.is_file() and entry.name.endswith(".csv"):
            df = pd.read_csv(entry.path, header=None, names=header)
            subset = df[df['TYPE'] == "populatedPlace"]
            results = pd.concat([results, subset[['NAME1', 'NAME2', 'GEOMETRY_X', 'GEOMETRY_Y']]])

/tmp/ipykernel_2413/3634243814.py:8: DtypeWarning: Columns (4,16,17,31,32,33) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(entry.path, header=None, names=header)
/tmp/ipykernel_2413/3634243814.py:8: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(entry.path, header=None, names=header)
/tmp/ipykernel_2413/3634243814.py:8: DtypeWarning: Columns (4,32,33) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(entry.path, header=None, names=header)
/tmp/ipykernel_2413/3634243814.py:8: DtypeWarning: Columns (4,31,33) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(entry.path, header=None, names=header)
/tmp/ipykernel_2413/3634243814.py:8: DtypeWarning: Columns (4,31,32,33) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(entry.path, header=None, nam

In [79]:
gdf = gpd.GeoDataFrame(results, geometry=gpd.points_from_xy(results['GEOMETRY_X'], results['GEOMETRY_Y'], crs=27700))
gdf = gdf.to_crs(4326)
gdf['lat'] = round(gdf.geometry.y, 3)
gdf['lon'] = round(gdf.geometry.x, 3)
gb_names = gdf[['NAME1', 'NAME2', 'lat', 'lon']].set_index(['lat', 'lon']).stack().reset_index(name='name').drop(columns='level_2').drop_duplicates()

Download the [OSNI Gazetteer](https://admin.opendatani.gov.uk/dataset/osni-open-data-gazetteer-place-names).

In [86]:
df = pd.read_csv('https://admin.opendatani.gov.uk/dataset/d27903f1-15e6-4c07-8564-ddc655e9c549/resource/411c6ea5-c079-43aa-bd62-438423d070cc/download/osni_open_data_-_gazetteer_-_place_names.csv')
igdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['X'], df['Y'], crs=29903))
igdf = igdf.to_crs(4326)
igdf['lat'] = round(igdf.geometry.y, 3)
igdf['lon'] = round(igdf.geometry.x, 3)
ni_names = igdf[['PLACENAME', 'lat', 'lon']].rename(columns={'PLACENAME': 'name'})
ni_names['name'] = ni_names['name'].str.title()

Download the [OpenStreetMap Names dataset](https://osmnames.org/), apply a bounding box and extract places. Then look up the places from Overpass to get Irish and English names.

In [3]:
planet = pd.read_table('https://github.com/OSMNames/OSMNames/releases/download/v2.0.4/planet-latest_geonames.tsv.gz', iterator=True, chunksize=100000, compression='gzip')
ireland = pd.concat([chunk[(chunk['lon'] > -9.97708574059) & (chunk['lon'] < -6.03298539878) & (chunk['lat'] > 51.6693012559) & (chunk['lat'] < 55.1316222195)] for chunk in planet])


/tmp/ipykernel_2413/2357205317.py:2: DtypeWarning: Columns (10,23) have mixed types. Specify dtype option on import or set low_memory=False.
  ireland = pd.concat([chunk[(chunk['lon'] > -9.97708574059) & (chunk['lon'] < -6.03298539878) & (chunk['lat'] > 51.6693012559) & (chunk['lat'] < 55.1316222195)] for chunk in planet])
/tmp/ipykernel_2413/2357205317.py:2: DtypeWarning: Columns (10,23) have mixed types. Specify dtype option on import or set low_memory=False.
  ireland = pd.concat([chunk[(chunk['lon'] > -9.97708574059) & (chunk['lon'] < -6.03298539878) & (chunk['lat'] > 51.6693012559) & (chunk['lat'] < 55.1316222195)] for chunk in planet])
/tmp/ipykernel_2413/2357205317.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  ireland = pd.concat([chunk[(chunk['lon'] > -9.97708574059) & (chunk['lon'] < -6.03298539878) & (chunk['lat'] > 51.6693012559) & (chunk['lat'] < 55.1316222195)] for chunk in planet])
/tmp/ipykernel_2413/23572053

In [64]:
places = ireland[(ireland['class'] == 'place') & (ireland['type'] != 'quarter') & (ireland['type'] != 'state')]
places.to_csv('osm-ireland-places.csv', index=False)
places.set_index('osm_id', inplace=True)

In [38]:
api = overpass.API(timeout=600)

In [48]:
def get_names(api, df, osm_type):
    idstr = ','.join(df[df['osm_type'] == osm_type]['osm_id'].astype(str).to_list())
    query = f"{osm_type}(id:{idstr});"
    elements = api.get(query, verbosity='tags', responseformat='json').get('elements')
    return {e['id']: {'ga': e.get('tags', {}).get('name:ga'), 'en': e.get('tags', {}).get('name:en'), 'name': e.get('tags', {}).get('name')} for e in elements}

relations = get_names(api, places, 'relation')


In [49]:
ways = get_names(api, places, 'way')

In [50]:
nodes = get_names(api, places, 'node')

In [51]:
nodes

{8563682: {'ga': 'Baile na mBocht', 'en': None, 'name': 'Mayfield'},
 24905097: {'ga': None, 'en': None, 'name': 'Garvary'},
 31078662: {'ga': None, 'en': None, 'name': 'Rosslare Harbour'},
 31086076: {'ga': 'Cora Droma Rúisc',
  'en': 'Carrick-on-Shannon',
  'name': 'Carrick-on-Shannon'},
 31241248: {'ga': None, 'en': None, 'name': 'Ballyleague'},
 32854991: {'ga': None, 'en': None, 'name': 'Ballinahown'},
 33583163: {'ga': None, 'en': None, 'name': 'Horseleap'},
 36681797: {'ga': None, 'en': None, 'name': 'Kilruane'},
 49305598: {'ga': 'Cluain Sceach', 'en': 'Clonskeagh', 'name': 'Clonskeagh'},
 51995744: {'ga': 'Baile Ui Ógain', 'en': 'Ballyogan', 'name': 'Ballyogan'},
 52218198: {'ga': 'Baile an Róba', 'en': 'Ballinrobe', 'name': 'Ballinrobe'},
 52218265: {'ga': 'An Fhianait', 'en': 'Fenit', 'name': 'Fenit'},
 52218266: {'ga': None, 'en': None, 'name': 'Bellewstown'},
 52218272: {'ga': 'Ráth Iomgháin', 'en': 'Rathangan', 'name': 'Rathangan'},
 52218281: {'ga': None, 'en': None, 'na

In [65]:
names = pd.DataFrame.from_dict(ways | nodes | relations, orient='index')
minimal = places.merge(names, left_index=True, right_index=True, how='left').reset_index()[['name_x', 'ga', 'en', 'lat', 'lon']]
minimal.rename(columns={'name_x': 'name'}, inplace=True)

In [ ]:
ireland_names = minimal.set_index(['lat', 'lon']).stack().reset_index(name='name').drop(columns='level_2').drop_duplicates()
ireland_names['lat'] = round(ireland_names.lat, 3)
ireland_names['lon'] = round(ireland_names.lon, 3)

In [ ]:
pd.concat([gb_names, ni_names, ireland_names])

,lat,lon,name
0,53.826,-1.221,Church Fenton
1,53.711,-1.198,Kellingley
2,53.720,-1.356,Half Acres
3,53.745,-1.324,Newton
4,53.728,-1.118,Chapel Haddlesey
...,...,...,...
5730,51.736,-9.543,Garnish Island
5731,52.474,-8.863,Ballingarry
5732,52.474,-8.863,Baile an Gharraí
5734,53.355,-6.281,Dublin


In [91]:
output = pd.concat([gb_names, ireland_names])[['name','lat','lon']].to_json(orient='values')
with open("app/places.json", "w") as fd:
    fd.write(output)